# 🌱 Creator Engine v7 — ESMFold pLDDT 검증

**UFoE 3-Layer Protein Design + Structure Validation**

| 한글 | 단백질 | Layer |
|------|--------|-------|
| 초성중성종성 | AA 타입 (곤감리건) | Layer 1: 재료 |
| 음절 | n-gram 패턴 | Layer 2a: 음절 |
| 단어 | 감-세그먼트-감 | Layer 2b: 단어 |
| 조사 | 리 접착 | Layer 2c: 조사 |
| 의미 | 기능 모티프 | Layer 3: 의미 |
| 문장 | 기능적 단백질 서열 | 완성 |

### 사용법
1. **런타임 → 런타임 유형 변경 → GPU (T4)** 선택
2. **런타임 → 모두 실행** (Ctrl+F9)
3. ESMFold 모델 로딩에 ~2분 소요


## 1. 패키지 설치

In [ ]:
!pip install torch transformers accelerate matplotlib pandas -q
print("✅ 패키지 설치 완료")


## 2. Creator Engine v7 코드

In [ ]:
import random
import json
import statistics
import time
import numpy as np
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Optional

random.seed(42)

# ════════════════════════════════════════
# 곤감리건 기본 정의
# ════════════════════════════════════════

AA_TABLE = {
    '곤': {'A': 0.15, 'L': 0.20, 'V': 0.18, 'I': 0.15, 'F': 0.12, 'M': 0.08, 'W': 0.05, 'P': 0.07},
    '감': {'G': 0.50, 'C': 0.15, 'P': 0.35},
    '리': {'S': 0.20, 'T': 0.18, 'N': 0.15, 'Q': 0.15, 'H': 0.12, 'Y': 0.20},
    '건': {'E': 0.30, 'D': 0.25, 'K': 0.25, 'R': 0.20},
}

AFFINITY = {
    ('곤','곤'): 3, ('곤','감'): 2, ('곤','리'): 2, ('곤','건'): 0,
    ('감','곤'): 2, ('감','감'): 1, ('감','리'): 2, ('감','건'): 1,
    ('리','곤'): 2, ('리','감'): 2, ('리','리'): 3, ('리','건'): 2,
    ('건','곤'): 0, ('건','감'): 1, ('건','리'): 2, ('건','건'): 2,
}

PROTEIN_TYPES = {
    'T1_Globular':    {'곤': 0.35, '감': 0.12, '리': 0.30, '건': 0.23},
    'T2_Enzyme':      {'곤': 0.32, '감': 0.10, '리': 0.33, '건': 0.25},
    'T3_Balanced':    {'곤': 0.30, '감': 0.15, '리': 0.30, '건': 0.25},
    'T4_MultiDomain': {'곤': 0.28, '감': 0.18, '리': 0.28, '건': 0.26},
    'T5_Special':     {'곤': 0.25, '감': 0.15, '리': 0.35, '건': 0.25},
}

# 모티프 라이브러리
MOTIF_LIBRARY = {
    'P-loop':           {'pattern': 'GxxxxGK',               'func': '곤', 'sig': '곤감', 'antag': False, 'min_len': 7},
    'Walker-B':         {'pattern': 'DxxG',                  'func': '곤', 'sig': '곤감', 'antag': False, 'min_len': 4},
    'Catalytic-triad':  {'pattern': 'HDS',                   'func': '건', 'sig': '곤리', 'antag': False, 'min_len': 3},
    'HTH':              {'pattern': 'xxLxxxGxxxx',           'func': '감', 'sig': '곤건', 'antag': True,  'min_len': 11},
    'Zinc-finger-C2H2': {'pattern': 'CxxCxxxxxxxxxxxxHxxxH', 'func': '리', 'sig': '혼합', 'antag': False, 'min_len': 21},
    'RGD':              {'pattern': 'RGD',                   'func': '리', 'sig': '건감', 'antag': False, 'min_len': 3},
}

MOTIF_SLOT_MAP = {
    'P-loop': 'ATP', 'Walker-B': 'ATP',
    'Catalytic-triad': 'Catalytic',
    'HTH': 'DNA',
    'Zinc-finger-C2H2': 'Zinc', 'RGD': 'Zinc',
}

print("✅ 곤감리건 정의 로드 완료")


## 3. 생성 엔진 (Layer 2a → 2b → 2c → 슬롯 → 의미)

In [ ]:
# ── Layer 2a: 감 리듬 생성기 (음절) ──

def generate_gam_rhythm(length, spacing_weights=None):
    if spacing_weights is None:
        spacing_weights = {1: 0.25, 2: 0.25, 3: 0.15, 4: 0.20, 5: 0.10, 8: 0.05}
    spacings = list(spacing_weights.keys())
    weights = list(spacing_weights.values())
    positions = [0]
    current = 0
    while current < length - 2:
        gap = random.choices(spacings, weights=weights)[0]
        next_pos = current + gap + 1
        if next_pos >= length:
            break
        positions.append(next_pos)
        current = next_pos
    return positions

# ── Layer 2b: 세그먼트 생성 (단어) ──

def generate_segments(length, gam_positions, composition):
    type_seq = ['?'] * length
    for pos in gam_positions:
        if pos < length:
            type_seq[pos] = '감'
    
    non_gam_total = composition.get('곤', 0.35) + composition.get('리', 0.30) + composition.get('건', 0.25)
    type_probs = {
        '곤': composition.get('곤', 0.35) / non_gam_total,
        '리': composition.get('리', 0.30) / non_gam_total,
        '건': composition.get('건', 0.25) / non_gam_total,
    }
    
    prev_type = '감'
    for i in range(length):
        if type_seq[i] != '?':
            prev_type = type_seq[i]
            continue
        transition_weights = {}
        for t in ['곤', '리', '건']:
            affinity = AFFINITY.get((prev_type, t), 1)
            base_prob = type_probs[t]
            self_bonus = 1.5 if t == prev_type else 1.0
            transition_weights[t] = base_prob * (affinity + 1) * self_bonus
        total_w = sum(transition_weights.values())
        probs = {t: w/total_w for t, w in transition_weights.items()}
        chosen = random.choices(list(probs.keys()), weights=list(probs.values()))[0]
        type_seq[i] = chosen
        prev_type = chosen
    return type_seq

# ── Layer 2c: 리 접착 강화 (조사) ──

def apply_ri_adhesion(type_seq):
    result = type_seq.copy()
    n = len(result)
    for i in range(1, n - 1):
        prev, cur, nxt = result[i-1], result[i], result[i+1]
        if prev == '곤' and nxt == '건' and cur not in ['감', '리']:
            if random.random() < 0.7:
                result[i] = '리'
        elif prev == '건' and nxt == '곤' and cur not in ['감', '리']:
            if random.random() < 0.7:
                result[i] = '리'
    return result

# ── Layer 2.5: 기능 슬롯 탐색 ──

def find_functional_slots(type_seq):
    n = len(type_seq)
    slots = {'ATP': [], 'Catalytic': [], 'DNA': [], 'Zinc': []}
    for i in range(3, n - 10):
        window = type_seq[max(0, i-3):min(n, i+4)]
        if len(window) < 5:
            continue
        transitions = sum(1 for a, b in zip(window, window[1:]) if a != b)
        density = transitions / (len(window) - 1)
        bigrams = [f"{window[j]}{window[j+1]}" for j in range(len(window)-1)]
        
        if '곤감' in bigrams and density > 0.6:
            slots['ATP'].append(i)
        gon_ri = sum(1 for bg in bigrams if bg in ['곤리', '리곤'])
        has_antag = any(bg in ['곤건', '건곤'] for bg in bigrams)
        if gon_ri >= 2 and not has_antag and density > 0.6:
            slots['Catalytic'].append(i)
        if has_antag and density > 0.5:
            slots['DNA'].append(i)
        if len(set(window)) >= 3 and density > 0.5:
            slots['Zinc'].append(i)
    return slots

# ── Layer 3: 의미 모티프 삽입 ──

def insert_motifs(type_seq, slots, requested_motifs):
    result = type_seq.copy()
    inserted = {}
    occupied = set()
    for motif_name in requested_motifs:
        if motif_name not in MOTIF_LIBRARY:
            continue
        motif = MOTIF_LIBRARY[motif_name]
        slot_key = MOTIF_SLOT_MAP.get(motif_name, 'ATP')
        available = slots.get(slot_key, [])
        candidates = [pos for pos in available
                      if not any(abs(pos - occ) < motif['min_len'] + 3 for occ in occupied)]
        if not candidates:
            continue
        best_pos = random.choice(candidates[:5])
        inserted[motif_name] = best_pos
        for j in range(motif['min_len']):
            if best_pos + j < len(result):
                occupied.add(best_pos + j)
    return result, inserted

# ── AA 선택기 ──

def select_amino_acids(type_seq, inserted_motifs, temperature=1.0):
    n = len(type_seq)
    sequence = ['X'] * n
    motif_positions = set()
    for motif_name, start_pos in inserted_motifs.items():
        motif = MOTIF_LIBRARY[motif_name]
        pattern = motif['pattern']
        for j, char in enumerate(pattern):
            pos = start_pos + j
            if pos >= n:
                break
            if char != 'x':
                sequence[pos] = char
                motif_positions.add(pos)
    for i in range(n):
        if i in motif_positions and sequence[i] != 'X':
            continue
        t = type_seq[i]
        if t not in AA_TABLE:
            t = '리'
        aa_choices = list(AA_TABLE[t].keys())
        aa_weights = list(AA_TABLE[t].values())
        if temperature != 1.0:
            aa_weights = [w ** (1.0 / temperature) for w in aa_weights]
        total = sum(aa_weights)
        aa_weights = [w / total for w in aa_weights]
        sequence[i] = random.choices(aa_choices, weights=aa_weights)[0]
    return ''.join(sequence)

# ── 자기검증 ──

def validate_sequence(sequence, type_seq, target_composition):
    n = len(sequence)
    actual_types = []
    for aa in sequence:
        found = False
        for t, aas in AA_TABLE.items():
            if aa in aas:
                actual_types.append(t)
                found = True
                break
        if not found:
            actual_types.append('?')
    type_counts = Counter(actual_types)
    actual_comp = {t: type_counts.get(t, 0) / n for t in ['곤', '감', '리', '건']}
    comp_error = sum(abs(actual_comp.get(t, 0) - target_composition.get(t, 0)) for t in ['곤', '감', '리', '건'])
    bigrams = Counter()
    for i in range(n - 1):
        bigrams[f"{actual_types[i]}{actual_types[i+1]}"] += 1
    trans_count = sum(1 for i in range(n-1) if actual_types[i] != actual_types[i+1])
    transition_density = trans_count / (n - 1) if n > 1 else 0
    return {
        'length': n, 'composition': actual_comp, 'composition_error': comp_error,
        'transition_density': transition_density,
        'direct_antagonism': bigrams.get('곤건', 0) + bigrams.get('건곤', 0),
        'top_bigrams': bigrams.most_common(5),
    }

# ── 전체 파이프라인 ──

def run_v7_pipeline(length=150, protein_type='T2_Enzyme',
                     requested_motifs=None, temperature=1.0, n_candidates=20):
    if requested_motifs is None:
        requested_motifs = ['P-loop', 'Catalytic-triad']
    composition = PROTEIN_TYPES[protein_type]
    results = []
    for trial in range(n_candidates):
        gam_positions = generate_gam_rhythm(length)
        type_seq = generate_segments(length, gam_positions, composition)
        type_seq = apply_ri_adhesion(type_seq)
        slots = find_functional_slots(type_seq)
        type_seq, inserted = insert_motifs(type_seq, slots, requested_motifs)
        sequence = select_amino_acids(type_seq, inserted, temperature)
        validation = validate_sequence(sequence, type_seq, composition)
        results.append({
            'trial': trial, 'sequence': sequence, 'type_seq': ''.join(type_seq),
            'inserted_motifs': inserted,
            'slots_found': {k: len(v) for k, v in slots.items()},
            'validation': validation,
        })
    results.sort(key=lambda r: r['validation']['composition_error'])
    return results

print("✅ Creator Engine v7 함수 로드 완료")


## 4. 서열 생성 (v7 + 대조군)

⚙️ 아래 `CONFIG`에서 설정을 변경할 수 있습니다.


In [ ]:
# ═══════════════════════════════════════
# ⚙️ 설정 (여기서 수정 가능)
# ═══════════════════════════════════════

CONFIG = {
    'length': 150,                           # 서열 길이
    'protein_type': 'T2_Enzyme',             # T1_Globular / T2_Enzyme / T3_Balanced / T4_MultiDomain / T5_Special
    'motifs': ['P-loop', 'Catalytic-triad'], # 삽입할 모티프
    'temperature': 1.0,                      # AA 선택 다양성 (0.5=보수적, 2.0=탐험적)
    'n_candidates': 20,                      # 후보 수
    'n_predict': 10,                         # ESMFold 예측할 서열 수 (그룹당)
}

# ═══════════════════════════════════════

print("=" * 70)
print("🌱 Creator Engine v7 — 서열 생성")
print("=" * 70)

# v7 서열 생성
v7_results = run_v7_pipeline(
    length=CONFIG['length'],
    protein_type=CONFIG['protein_type'],
    requested_motifs=CONFIG['motifs'],
    temperature=CONFIG['temperature'],
    n_candidates=CONFIG['n_candidates'],
)

# 랜덤 대조군
def generate_random_control(length, n=20):
    all_aa = 'ACDEFGHIKLMNPQRSTVWY'
    return [{'trial': i, 'sequence': ''.join(random.choices(all_aa, k=length)), 'label': f'random_{i}'} for i in range(n)]

# 조성 매칭 대조군
def generate_composition_control(length, composition, n=20):
    controls = []
    for trial in range(n):
        type_list = []
        for t, ratio in composition.items():
            type_list.extend([t] * int(length * ratio))
        while len(type_list) < length:
            type_list.append('리')
        random.shuffle(type_list)
        seq = [random.choice(list(AA_TABLE.get(t, AA_TABLE['리']).keys())) for t in type_list]
        controls.append({'trial': trial, 'sequence': ''.join(seq), 'label': f'comp_ctrl_{trial}'})
    return controls

random_controls = generate_random_control(CONFIG['length'], n=CONFIG['n_candidates'])
comp_controls = generate_composition_control(CONFIG['length'], PROTEIN_TYPES[CONFIG['protein_type']], n=CONFIG['n_candidates'])

print(f"\n생성 완료:")
print(f"  v7 후보: {len(v7_results)}개")
print(f"  랜덤 대조군: {len(random_controls)}개")
print(f"  조성 대조군: {len(comp_controls)}개")

print(f"\n▼ v7 상위 5개 서열")
for r in v7_results[:5]:
    v = r['validation']
    motif_str = ', '.join(f"{k}@{v}" for k, v in r['inserted_motifs'].items()) if r['inserted_motifs'] else 'none'
    print(f"  Trial {r['trial']:2d} | err={v['composition_error']:.3f} | trans={v['transition_density']:.3f} | motifs: {motif_str}")
    print(f"           {r['sequence'][:70]}...")


## 5. ESMFold 모델 로딩

⏳ 첫 실행 시 모델 다운로드에 ~2분 소요됩니다.


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")
else:
    print("⚠️ GPU를 사용할 수 없습니다. 런타임 → 런타임 유형 변경 → GPU 선택해주세요.")

print("\n⏳ ESMFold 모델 로딩 중...")
from transformers import EsmForProteinFolding, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1", low_cpu_mem_usage=True)
model = model.to(device)
model.eval()

if device.type == 'cuda':
    model.trunk.set_chunk_size(64)

print("✅ ESMFold 로드 완료!")


## 6. pLDDT 예측 실행

In [ ]:
def predict_plddt(sequence, model, tokenizer, device):
    try:
        with torch.no_grad():
            tokenized = tokenizer([sequence], return_tensors="pt", add_special_tokens=False).to(device)
            output = model(**tokenized)
            plddt = output['plddt'][0, :, 1]
            mean_plddt = plddt.mean().item() * 100
            per_residue = (plddt * 100).cpu().numpy()
            return {
                'mean_plddt': mean_plddt,
                'per_residue': per_residue,
                'max_plddt': per_residue.max(),
                'min_plddt': per_residue.min(),
                'plddt_gt50': (per_residue > 50).sum() / len(per_residue) * 100,
                'plddt_gt70': (per_residue > 70).sum() / len(per_residue) * 100,
            }
    except Exception as e:
        print(f"  ⚠️ 에러: {e}")
        return {'mean_plddt': 0, 'per_residue': np.zeros(len(sequence)),
                'max_plddt': 0, 'min_plddt': 0, 'plddt_gt50': 0, 'plddt_gt70': 0}

n_pred = CONFIG['n_predict']

sequences_to_predict = {
    'v7_UFoE': [{'trial': r['trial'], 'sequence': r['sequence']} for r in v7_results[:n_pred]],
    'Random_Control': random_controls[:n_pred],
    'Composition_Control': comp_controls[:n_pred],
}

plddt_results = {}

for group_name, sequences in sequences_to_predict.items():
    print(f"\n📊 {group_name} ({len(sequences)}개)...")
    group_results = []
    
    for i, item in enumerate(sequences):
        valid_aa = set('ACDEFGHIKLMNPQRSTVWY')
        seq_clean = ''.join(c for c in item['sequence'] if c in valid_aa)
        if len(seq_clean) < 20:
            continue
        
        result = predict_plddt(seq_clean, model, tokenizer, device)
        result['trial'] = item.get('trial', i)
        result['sequence'] = seq_clean
        group_results.append(result)
        
        print(f"  [{i+1:2d}/{len(sequences)}] pLDDT={result['mean_plddt']:5.1f} | "
              f">50: {result['plddt_gt50']:5.1f}% | >70: {result['plddt_gt70']:5.1f}%")
        
        if device.type == 'cuda':
            torch.cuda.empty_cache()
    
    plddt_results[group_name] = group_results

print("\n✅ 예측 완료!")


## 7. 결과 분석

In [ ]:
print("=" * 70)
print("📊 결과 분석")
print("=" * 70)

print(f"\n{'그룹':<25} {'N':<5} {'평균pLDDT':<12} {'최대':<8} {'최소':<8} {'>50%':<8} {'>70%':<8}")
print(f"{'─' * 75}")

summary = {}
for group_name, results in plddt_results.items():
    if not results:
        continue
    plddts = [r['mean_plddt'] for r in results]
    gt50s = [r['plddt_gt50'] for r in results]
    gt70s = [r['plddt_gt70'] for r in results]
    
    mean_p = statistics.mean(plddts)
    summary[group_name] = {
        'mean': mean_p,
        'max': max(plddts),
        'min': min(plddts),
        'std': statistics.stdev(plddts) if len(plddts) > 1 else 0,
        'gt50': statistics.mean(gt50s),
        'gt70': statistics.mean(gt70s),
        'all_plddts': plddts,
    }
    print(f"{group_name:<25} {len(results):<5} {mean_p:<12.1f} {max(plddts):<8.1f} "
          f"{min(plddts):<8.1f} {statistics.mean(gt50s):<8.1f} {statistics.mean(gt70s):<8.1f}")

# 비교
print(f"\n{'─' * 75}")
if 'v7_UFoE' in summary and 'Random_Control' in summary:
    diff = summary['v7_UFoE']['mean'] - summary['Random_Control']['mean']
    mark = "✅" if diff > 5 else ("△" if diff > 0 else "✗")
    print(f"v7 vs Random:     {diff:+.1f} pLDDT {mark}")

if 'v7_UFoE' in summary and 'Composition_Control' in summary:
    diff = summary['v7_UFoE']['mean'] - summary['Composition_Control']['mean']
    mark = "✅" if diff > 5 else ("△" if diff > 0 else "✗")
    print(f"v7 vs Composition: {diff:+.1f} pLDDT {mark} ← 패턴(Layer2)+의미(Layer3) 효과")


## 8. 시각화

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.size'] = 11

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
colors_map = {'v7_UFoE': '#2196F3', 'Random_Control': '#FF5722', 'Composition_Control': '#4CAF50'}

# (1) 박스플롯
ax1 = axes[0, 0]
groups = [s['all_plddts'] for s in summary.values()]
labels = [n.replace('_', '\n') for n in summary.keys()]
bp = ax1.boxplot(groups, labels=labels, patch_artist=True)
for patch, gname in zip(bp['boxes'], summary.keys()):
    patch.set_facecolor(colors_map.get(gname, '#999'))
    patch.set_alpha(0.7)
ax1.set_ylabel('Mean pLDDT')
ax1.set_title('pLDDT Distribution by Group')
ax1.axhline(y=50, color='orange', linestyle='--', alpha=0.5, label='pLDDT=50')
ax1.axhline(y=70, color='green', linestyle='--', alpha=0.5, label='pLDDT=70')
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# (2) 히스토그램
ax2 = axes[0, 1]
for gname, s in summary.items():
    ax2.hist(s['all_plddts'], bins=10, alpha=0.5, label=gname, color=colors_map.get(gname, '#999'))
ax2.set_xlabel('Mean pLDDT')
ax2.set_ylabel('Count')
ax2.set_title('pLDDT Histogram')
ax2.legend()
ax2.grid(alpha=0.3)

# (3) per-residue pLDDT (v7 Top 3)
ax3 = axes[1, 0]
v7_group = plddt_results.get('v7_UFoE', [])
for r in sorted(v7_group, key=lambda x: -x['mean_plddt'])[:3]:
    ax3.plot(r['per_residue'], alpha=0.7, label=f"v7 #{r['trial']} (avg={r['mean_plddt']:.1f})")
ax3.axhline(y=50, color='orange', linestyle='--', alpha=0.5)
ax3.axhline(y=70, color='green', linestyle='--', alpha=0.5)
ax3.set_xlabel('Residue Position')
ax3.set_ylabel('pLDDT')
ax3.set_title('Per-Residue pLDDT (v7 Top 3)')
ax3.legend(fontsize=8)
ax3.grid(alpha=0.3)

# (4) 조성 오차 vs pLDDT
ax4 = axes[1, 1]
if v7_group:
    trial_to_comp_err = {r['trial']: r['validation']['composition_error'] for r in v7_results}
    comp_errors = [trial_to_comp_err.get(r['trial'], 0) for r in v7_group]
    plddts_v7 = [r['mean_plddt'] for r in v7_group]
    ax4.scatter(comp_errors, plddts_v7, c='#2196F3', alpha=0.7, s=80)
    ax4.set_xlabel('Composition Error')
    ax4.set_ylabel('Mean pLDDT')
    ax4.set_title('Composition Error vs pLDDT (v7)')
    ax4.grid(alpha=0.3)
    if len(comp_errors) > 2:
        z = np.polyfit(comp_errors, plddts_v7, 1)
        p = np.poly1d(z)
        x_range = np.linspace(min(comp_errors), max(comp_errors), 50)
        ax4.plot(x_range, p(x_range), 'r--', alpha=0.5, label=f'slope={z[0]:.1f}')
        ax4.legend()

plt.tight_layout()
plt.savefig('v7_plddt_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("📊 그래프 저장: v7_plddt_results.png")


## 9. 최종 판정

In [ ]:
print("=" * 70)
print("📋 Creator Engine v7 — 최종 판정")
print("=" * 70)

v7_s = summary.get('v7_UFoE', {})
if v7_s:
    mean_p = v7_s['mean']
    
    if mean_p >= 70:
        verdict = "🎉 성공: 안정적 단백질 구조 생성 확인"
    elif mean_p >= 50:
        verdict = "✅ 유망: 부분적 구조 형성, 개선 여지 있음"
    elif mean_p >= 30:
        verdict = "△ 진행 중: 구조 경향 있으나 추가 최적화 필요"
    else:
        verdict = "⚠️ 미달: 기본 패턴 재검토 필요"
    
    print(f"\n  v7 평균 pLDDT: {mean_p:.1f} ± {v7_s['std']:.1f}")
    print(f"  >50 비율: {v7_s['gt50']:.1f}%")
    print(f"  >70 비율: {v7_s['gt70']:.1f}%")
    print(f"\n  → {verdict}")

# JSON 저장
final_output = {
    'config': CONFIG,
    'summary': {k: {sk: sv for sk, sv in v.items() if sk != 'all_plddts'} for k, v in summary.items()},
    'v7_top5': [{
        'trial': r['trial'], 'sequence': r['sequence'],
        'mean_plddt': next((pr['mean_plddt'] for pr in plddt_results.get('v7_UFoE', []) if pr['trial'] == r['trial']), None),
        'comp_error': r['validation']['composition_error'],
        'motifs': r['inserted_motifs'],
    } for r in v7_results[:5]],
}
with open('v7_final_report.json', 'w') as f:
    json.dump(final_output, f, ensure_ascii=False, indent=2)

print(f"\n💾 저장: v7_final_report.json")
print(f"\n✅ Creator Engine v7 검증 완료!")
